In [1]:
import pennylane as qml
from pennylane import numpy as np
import torch
from math import pi

In [2]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable

class makedata(Dataset):
    def __init__(self, col, n_feat, maximum):
        self.col = col  #number of datapoints
        self.n_feat = n_feat  #number of features
        self.maximum = maximum  #increase the range of feature data
        self.classes = 2   #number of classes

        if col%2 != 0:
            col = col+1
            print("Making datasize an even number for ease: col =", col)
            
        self.Xdata1 = Variable(torch.rand(col//2, n_feat)*maximum -0.4 , requires_grad=True) 
        self.Xdata2 = Variable(torch.rand(col//2, n_feat)*(-maximum) +0.4, requires_grad=True)
        self.Xdata = torch.cat((self.Xdata1, self.Xdata2), dim=0)

        self.Ydata = torch.cat((torch.full((col//2,), 1),torch.full((col//2,), -1)))       

    def __len__(self):
        return len(self.Ydata)

    def __getitem__(self, idx):
        return self.Xdata[idx],self.Ydata[idx]


In [3]:
r =5
data = makedata(100, 2, r)

In [4]:
print(len(data))
print(data.Xdata1)
print(data.Xdata2)
print(data.Xdata)
print(data.Ydata)
x,y = data.__getitem__(2)
print(x,y)

100
tensor([[ 3.5271e+00,  7.9728e-01],
        [ 2.9652e+00,  1.3445e+00],
        [ 2.2188e+00,  4.5787e+00],
        [ 2.7827e+00,  3.4060e+00],
        [ 6.3849e-01, -3.6385e-01],
        [ 3.1823e+00,  3.8951e+00],
        [ 3.6600e+00,  2.2477e+00],
        [ 2.3494e+00,  6.9114e-01],
        [ 2.3536e-01,  4.3942e+00],
        [ 2.2289e+00,  1.5881e+00],
        [ 7.2946e-01,  1.8004e+00],
        [ 3.3213e+00,  1.9918e+00],
        [ 2.8237e-01,  1.0404e+00],
        [ 3.5336e-01,  1.0284e+00],
        [ 4.0568e+00,  2.1841e+00],
        [ 3.0561e-03,  3.5345e+00],
        [ 1.0025e+00,  1.7504e+00],
        [ 2.5314e+00,  3.3267e+00],
        [ 2.2823e+00,  3.3761e+00],
        [ 7.6741e-01,  3.3343e+00],
        [ 1.4858e-01,  6.5780e-01],
        [ 7.0094e-01,  1.4629e+00],
        [ 4.0152e+00,  2.1622e+00],
        [ 4.0587e+00, -2.5349e-01],
        [ 2.3039e+00, -1.2409e-01],
        [ 1.2279e+00,  1.9639e+00],
        [ 1.6273e+00,  3.6636e-01],
        [ 1.1120e+00,  1

In [23]:
batch_size = 2
data_batches = DataLoader(data, batch_size=batch_size, shuffle=True)
for d in data_batches:
    x,y = d
    print(x,y)
    break
print(x.shape[0])

tensor([[-3.6916, -1.9226],
        [-1.9381,  0.1062]], grad_fn=<StackBackward>) tensor([-1., -1.])
2


In [6]:
np_Xdata1 = data.Xdata1.detach().numpy()
np_Xdata2 = data.Xdata2.detach().numpy()

In [7]:
import matplotlib.pyplot as plt

colors = ("red", "blue")
groups = ("class 1", "class 2")

plt.scatter(np_Xdata1[:,0], np_Xdata1[:,1], color="tab:red", label= "Class1")
plt.scatter(np_Xdata2[:,0], np_Xdata2[:,1], color="tab:blue", label = "Class2")
plt.legend(loc = 'best')
plt.show()

<Figure size 640x480 with 1 Axes>

In [8]:
from pennylane.templates.embeddings import AmplitudeEmbedding

def randlayer(W):
    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
    qml.CNOT(wires=[0,1])

dev = qml.device('default.qubit', wires=2)

@qml.qnode(dev, interface = 'torch')
def circuit(w1, w2, p=None):
    AmplitudeEmbedding(p, wires=[0,1], pad=True, normalize=True)
    randlayer(w1)
    randlayer(w2)
    return qml.expval(qml.PauliZ(0))


In [24]:
def get_pred(w1, w2, x):   #predictions for one batch
    n_points= x.shape[0]
    pred_batch = torch.empty(n_points,)    
    for i in range(n_points):
        pred = circuit(w1, w2, p=x[i])
        if pred > 0:
            torch.ceil(pred)
        elif pred < 0:
            torch.floor(pred)
        pred_batch[i]=pred
    return pred_batch

In [10]:
def get_loss(w1, w2, x, y):
    loss = torch.empty(1,)
    pred = get_pred(w1, w2, x)
    loss = torch.sum((pred-y)**2)
    return loss/len(y)

In [11]:
torch.manual_seed(1)
w1_init = Variable((2*pi * torch.rand((2, 3), dtype=torch.float64)), requires_grad=True)
w2_init = Variable((2*pi * torch.rand((2, 3), dtype=torch.float64)), requires_grad=True)

In [ ]:
test = Variable(torch.tensor([0.001, 0.004]),requires_grad = True)
res = circuit(w1_init, w2_init, p=test)
print(res)

In [ ]:
res.backward()
w1_init.grad

In [12]:
w1= w1_init
w2= w2_init

In [15]:
opt = torch.optim.SGD([w1, w2], lr = 0.01)

for epoch in range(3):
    for d in data_batches:
        x,y = d
        rec= []
        for step in range(300):
            opt.zero_grad()
            loss = get_loss(w1, w2, x, y)
            loss.backward()
            opt.step()
            rec.append(loss.item())

KeyboardInterrupt: 

In [18]:
fin1, fin2 = opt.param_groups[0]['params']
print(fin1)

tensor([[0.3836, 2.2052, 0.8695],
        [0.6823, 4.7685, 1.7008]], dtype=torch.float64, requires_grad=True)


In [25]:
with torch.no_grad():
    x,y = data.__getitem__(2)    
    pred = get_pred(fin1,fin2,x)
    print(pred, y)

TypeError: len() of unsized object

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Classifier(nn.Module):
    
    def __init__(self):
        super(Classifier, self).__init__()
        self.w1_init = Variable((2*pi * torch.rand((2, 3), dtype=torch.float64)), requires_grad=True)
        self.w2_init = Variable((2*pi * torch.rand((2, 3), dtype=torch.float64)), requires_grad=True)
        
    def forward(self, w1, w2, x):  #what to do with data in these layers
        @qml.qnode(dev, interface = 'torch')
        def circuit(w1, w2, p=None):
            AmplitudeEmbedding(p, wires=[0,1], pad=True, normalize=True)
            randlayer(w1)
            randlayer(w2)
            return qml.expval(qml.PauliZ(0))
        pred = circuit(w1, w2, p=x)
        if pred > 0:
            return torch.ceil(pred)
        elif pred < 0:
            return torch.floor(pred)
    

In [ ]:
test_cl = Classifier()
print(test_cl.w1_init)

In [ ]:
out = test_classifier(test)
out = out.view(1,)
print(out.view(1,))
target = Variable(torch.tensor([1], dtype=torch.float64),requires_grad = True)
print(target)

In [ ]:
loss = F.mse_loss(out, target)
print(loss)

In [ ]:
opt = torch.optim.Adam([test_cl.w1_init,test_cl.w2_init], lr = 0.01)

for epoch in range(3):
    for d in data_batches:
        x,y = d
        opt.zero_grad()
        out = test_classifier(x)
        out = out.view(1,)
        loss = F.mse_loss(out, y)
        loss.backward()
        opt.step()
    print(loss)

In [ ]:
tp = torch.tensor([1.0, 1.0])
ty= torch.tensor([-1.0, 1.0])
print(torch.sum((tp-ty)**2))